# Caching function results
This notebook demonstrates how one can use the MLRepo to cache function calls. Especially when working interactively using jupyter notebooks it may be usefull to 
cache function values for time consuming functions. In this notebook we demonstrate how to use pailab's ml_cache decorator to realize caching for functions.
## Setup

In [1]:
import numpy as np
from pailab.ml_repo.repo import MLRepo, MLObjectType
from pailab.ml_repo.repo_objects import RawData, RepoInfoKey, RepoObject

import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
ml_repo = MLRepo(user='job_runner_user')

For demonstration we use a simple example: We just create two RawData objects, one holding input data of a function evaluated at random points and one artificially created model output (where we just add a andom number to the function values to simulate some kind of model output with error).

In [3]:
x = np.random.rand(10000, 3)
y = x[:,1]*x[:,2]+ x[:,0]
y_approx =y + x[:,1]*np.random.rand(10000)
data = RawData(x, ['x0', 'x1', 'x2'], y, ['f'], repo_info={RepoInfoKey.NAME: 'eval', RepoInfoKey.CATEGORY: MLObjectType.TRAINING_DATA})
data_eval = RawData(y_approx, ['eval'],  repo_info={RepoInfoKey.NAME: 'error', RepoInfoKey.CATEGORY: MLObjectType.EVAL_DATA})

In [4]:
ml_repo.add([data, data_eval])

{'error': '90b42d76-6978-11e9-b8c6-fc084a6691eb',
 'eval': '90b42d76-6978-11e9-b8c6-fc084a6691eb'}

## Using the ml_cache decorator

We define a simple function computing the pointwise distance from the model function values to the input function values. To show what can be cached the function returns  a string, a double value, a numpy array and the numpy array encapsulated in a RawData object.
A print statement shows if the function has been executed. We use the ml_cache decorator to avoid evaluations if inputs have not been changed.

In [5]:
from pailab.tools.tools import ml_cache

@ml_cache
def eval_error(input_data, eval_data, factor):
    error = input_data.y_data-eval_data.x_data
    error_repo = RawData(error, ['error'], repo_info={RepoInfoKey.NAME: 'error', RepoInfoKey.CATEGORY: MLObjectType.CACHED_VALUE})
    error_mean = error.mean()
    print('Has been evaluated!')
    return 'super', factor*error_mean, y, error_repo, error

We evaluate the function for the first time.

In [6]:
results = eval_error(ml_repo, data, data_eval, 1.0)
print(results)

Has been evaluated!
('super', -0.25162381542531698, array([ 1.19037203,  0.31351727,  1.03094508, ...,  0.56388649,
        0.31437233,  0.82683433]), <pailab.ml_repo.repo_objects.RawData object at 0x00000116E79A9048>, array([[-0.05533983],
       [-0.00442759],
       [-0.0677045 ],
       ..., 
       [-0.38789236],
       [-0.18058904],
       [-0.00588409]]))


If we call the function again using the same arguments, the function is not executed, but results are returned.

In [7]:
results = eval_error(ml_repo, data, data_eval, 1.0)
print(results)

('super', -0.25162381542531698, array([ 1.19037203,  0.31351727,  1.03094508, ...,  0.56388649,
        0.31437233,  0.82683433]), <pailab.ml_repo.repo_objects.RawData object at 0x00000116EA21D518>, array([[-0.05533983],
       [-0.00442759],
       [-0.0677045 ],
       ..., 
       [-0.38789236],
       [-0.18058904],
       [-0.00588409]]))


Now we call the function with a modified argument which leads to a new function evaluatio.

In [8]:
results = eval_error(ml_repo, data, data_eval, 2.0)
print(results)

Has been evaluated!
('super', -0.50324763085063395, array([ 1.19037203,  0.31351727,  1.03094508, ...,  0.56388649,
        0.31437233,  0.82683433]), <pailab.ml_repo.repo_objects.RawData object at 0x00000116EA21D320>, array([[-0.05533983],
       [-0.00442759],
       [-0.0677045 ],
       ..., 
       [-0.38789236],
       [-0.18058904],
       [-0.00588409]]))


Using the previous argument returns the previous result without evaluating the function.

In [9]:
results = eval_error(ml_repo, data, data_eval, 1.0)
print(results)

('super', -0.25162381542531698, array([ 1.19037203,  0.31351727,  1.03094508, ...,  0.56388649,
        0.31437233,  0.82683433]), <pailab.ml_repo.repo_objects.RawData object at 0x00000116EA21D5F8>, array([[-0.05533983],
       [-0.00442759],
       [-0.0677045 ],
       ..., 
       [-0.38789236],
       [-0.18058904],
       [-0.00588409]]))
